In [46]:
import tensorflow as tf
#import tensorflow.contrib.eager as tfe
import os
import matplotlib.pyplot as plt
import re
import numpy as np
from string import punctuation
from collections import defaultdict
from functools import reduce
from keras.preprocessing.sequence import pad_sequences
from itertools import chain
from InputPreparator import EmbeddingsPreparator
from InputPreparator import StoryParser
import time

#to avoid a warning from TF 1.7 version see https://github.com/tensorflow/tensorflow/issues/18111
import warnings
warnings.filterwarnings('ignore')


# Input preparation

In [4]:
def get_task_files(task_nr):
    if task_nr==5:
        return 'qa5_three-arg-relations_train.txt', "qa5_three-arg-relations_test.txt"
    if task_nr==6:
        return 'qa6_yes-no-questions_train.txt', 'qa6_yes-no-questions_test.txt'
    if task_nr==10:
        return 'qa10_indefinite-knowledge_train.txt', 'qa10_indefinite-knowledge_test.txt'

In [26]:
train_set_file = get_task_files(5)[0]
test_set_file = get_task_files(5)[0]

train_set_post_file = "data/tasks_1-20_v1-2/en/"+train_set_file
test_set_post_file = "data/tasks_1-20_v1-2/en/"+test_set_file

In [27]:
embedder=EmbeddingsPreparator()
story_parser=StoryParser()

In [28]:
vocab_tokens = embedder.get_unique_tokens([train_set_post_file, test_set_post_file])

In [29]:
word_to_index, index_to_embedding = embedder.load_embedding_from_disks("data/glove.6B.50d.txt",vocab_tokens, with_indexes=True)

In [30]:
train_stories=story_parser.get_stories(train_set_post_file, True)
test_stories=story_parser.get_stories(test_set_post_file, True)

In [31]:
contexts_train, questions_train, answers_train = story_parser.vectorize_stories(train_stories, word_to_index)
contexts_test, questions_test, answers_test = story_parser.vectorize_stories(test_stories, word_to_index)

In [32]:
print('contexts.shape = {}'.format(contexts_train.shape))
print('questions.shape = {}'.format(questions_train.shape))
print('answers.shape = {}'.format(answers_train.shape))

contexts.shape = (1000, 7)
questions.shape = (1000,)
answers.shape = (1000, 42)


In [33]:
index_to_embedding.shape[1]

50

In [34]:
final_train_data = story_parser.get_final_dataset(contexts_train, questions_train, answers_train)
final_test_data = story_parser.get_final_dataset(contexts_test, questions_test, answers_test)

In [40]:
def prep_data(data, all_data= False, train= False):
    contextsvs, questionsvs, answers=zip(*data)
    
    #Pad to longest sequence in the batch 
    contexts = list(contextsvs)
    max_context_length = max([len(x) for x in contexts])
    questions = list(questionsvs)
    max_query_length = max(len(x) for x in questionsvs)

    final_contexts=pad_sequences(contextsvs, maxlen=max_context_length) 
    final_queries=pad_sequences(questionsvs, maxlen=max_query_length)
    
    return final_contexts, final_queries

# Model

In [36]:
final_contexts, final_queries= prep_data(final_train_data)

vocab_size= len(index_to_embedding)
embed_dimensions= 50
num_units_gru= 50
keep_prob= 0.5
q_shape= final_queries.shape[1]
c_shape= final_contexts.shape[1]

In [37]:
q_in = tf.keras.Input(shape=(q_shape,))
c_in = tf.keras.Input(shape=(c_shape,))

#embed
embed = tf.keras.layers.Embedding(vocab_size, embed_dimensions)
q = embed(q_in)
c = embed(c_in)

#encode
grucell= tf.keras.layers.GRUCell(num_units_gru)
c = tf.keras.layers.RNN(grucell)(c)
q = tf.keras.layers.RNN(grucell)(q)

#add dropout
dropout= tf.keras.layers.Dropout(keep_prob)
encoded_sentence= dropout(c)
encoded_question= dropout(q)

#merge
merged= tf.keras.layers.concatenate([encoded_sentence, encoded_question])

#predict
pred=tf.keras.layers.Dense(answers_train.shape[1], activation=tf.nn.softmax)(merged)

model = tf.keras.Model(inputs=[q_in, c_in], outputs=pred)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 7)            0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 8)            0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         multiple             2100        input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
rnn_3 (RNN)                     (None, 50)           15150       embedding_2[1][0]                
__________

## Train

In [47]:

print('Training...')
start_time = time.time()
model.fit([final_queries, final_contexts], answers_train, epochs=100, batch_size=128)
elapsed_time = time.time() - start_time
print()
print('Training time: ')
print(elapsed_time)

Training...
Epoch 1/100
1000/1000 [==============================] - 0s 196us/step - loss: 0.3308 - acc: 0.8550
Epoch 2/100
1000/1000 [==============================] - 0s 210us/step - loss: 0.3542 - acc: 0.8370
Epoch 3/100
1000/1000 [==============================] - 0s 188us/step - loss: 0.3408 - acc: 0.8430
Epoch 4/100
1000/1000 [==============================] - 0s 199us/step - loss: 0.3298 - acc: 0.8600
Epoch 5/100
1000/1000 [==============================] - 0s 187us/step - loss: 0.3355 - acc: 0.8540
Epoch 6/100
1000/1000 [==============================] - 0s 203us/step - loss: 0.3283 - acc: 0.8560
Epoch 7/100
1000/1000 [==============================] - 0s 209us/step - loss: 0.3219 - acc: 0.8630
Epoch 8/100
1000/1000 [==============================] - 0s 192us/step - loss: 0.3386 - acc: 0.8590
Epoch 9/100
1000/1000 [==============================] - 0s 200us/step - loss: 0.3178 - acc: 0.8600
Epoch 10/100
1000/1000 [==============================] - 0s 202us/step - loss: 0.3123 -

1000/1000 [==============================] - 0s 172us/step - loss: 0.2724 - acc: 0.8860
Epoch 82/100
1000/1000 [==============================] - 0s 170us/step - loss: 0.3007 - acc: 0.8650
Epoch 83/100
1000/1000 [==============================] - 0s 173us/step - loss: 0.3098 - acc: 0.8660
Epoch 84/100
1000/1000 [==============================] - 0s 168us/step - loss: 0.2899 - acc: 0.8740
Epoch 85/100
1000/1000 [==============================] - 0s 177us/step - loss: 0.3059 - acc: 0.8630
Epoch 86/100
1000/1000 [==============================] - 0s 168us/step - loss: 0.2826 - acc: 0.8680
Epoch 87/100
1000/1000 [==============================] - 0s 189us/step - loss: 0.2711 - acc: 0.8810
Epoch 88/100
1000/1000 [==============================] - 0s 169us/step - loss: 0.2916 - acc: 0.8700
Epoch 89/100
1000/1000 [==============================] - 0s 190us/step - loss: 0.2909 - acc: 0.8670
Epoch 90/100
1000/1000 [==============================] - 0s 184us/step - loss: 0.2801 - acc: 0.8740
Epo

# Test

In [48]:
final_context_test, final_queries_test= prep_data(final_test_data)

In [49]:
score = model.evaluate([final_queries_test,final_context_test], answers_test, batch_size=128)

1000/1000 [==============================] - 0s 79us/step


In [50]:
score

[0.20600797593593598, 0.8820000004768371]